<a href="https://colab.research.google.com/github/Herais/MicrobialAnalysis/blob/main/MBA_antecedentSearchByLift.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prepare environment

In [ ]:
import torch
print(torch.cuda.is_available())
print(torch.__version__)

True
2.0.1+cu118


In [ ]:
import pandas as pd
import numpy as np
import re
import gc
import itertools
from itertools import zip_longest

from google.colab import files

from scipy.stats import norm
#from multitest import MultiTest
import gdown
import os
import pickle
from collections import Counter
from collections import OrderedDict

import scipy
from scipy.stats import fisher_exact
from scipy.stats.contingency import odds_ratio

import statsmodels
from statsmodels.stats.multitest import multipletests

#import markov_clustering as mc
import networkx as nx
import random
import math

# Import and preprocess data

In [ ]:
fp_data_ccsf = '/content/OTU_CF-soil_particle.csv'
id_file_ccsf = '1gc66lzz7kalmgmPaGxx5coDkT1H58YuA'
gdown.download(id=id_file_ccsf, output=fp_data_ccsf, quiet=True)

'/content/OTU_CF-soil_particle.csv'

In [ ]:
fp_data_all = '/content/all_otu_counts.ssp.csv'
id_file_all = '1YfT1oJD9ZVZk81Q_XQPzm1NifftslRna'
gdown.download(id=id_file_all, output=fp_data_all, quiet=True)

'/content/all_otu_counts.ssp.csv'

In [ ]:
df_all = pd.read_csv(fp_data_all)

In [ ]:
df_all['site'] = df_all['Unnamed: 0'].apply(lambda x: x[:2])
df_all['site'].unique()

array(['CF', 'HB', 'KB', 'Pn', 'UB', 'UF', 'WM'], dtype=object)

In [ ]:
df = df_all
cols = df.columns
cols = [col for col in cols if not re.search('Unnamed: 0|site', col)]
cols = sorted(cols, key=lambda x: int(x[3:]))

# assign presence/absence
dft = df[cols].gt(0, axis=0)
dft = dft.astype('int')

n_cooccur_max = dft.to_numpy().sum(axis=1).max()
print(n_cooccur_max)

163


# CHANGE OTU NAME HERE

In [ ]:
MIN_COUNT = 3
N_POOL = 4
BATCH_SIZE = 5000


############
OTU_NAME = 'Otu11'
############

In [ ]:
# only keep events that has otu of interest
df_filtered =  dft[dft[OTU_NAME] > 0].copy()
df_filtered['sid'] = df_all['Unnamed: 0']

print('# of events: {}'.format(df.shape[0]))
#print('# of events after filtering for Otu1 with events: {}'.format(df_filtered.shape[0]))

# drop all Otu without any events
print('# of OTU: {}'.format(df_filtered.shape[1] -1))

tmp = df_filtered[cols].apply(sum, axis=0) > 0
col_filtered = list(tmp[tmp].index)

df_filtered = df_filtered[['sid'] + col_filtered]
print('# of OTU after filtering for OTU with events: {}'.format(df_filtered.shape[1] -1))

# of events: 4049
# of OTU: 13299
# of OTU after filtering for OTU with events: 62


In [ ]:
colnames = [colname for colname in df_filtered.columns if (colname not in ('sid', OTU_NAME)) ]

transactions = df_filtered[colnames]
a_otu = np.array(list(transactions.columns))
basket = transactions.astype('bool').apply(lambda x: a_otu[x], axis=1)
ls_refset = basket.apply(set).tolist()

In [ ]:
df1 = transactions.loc[:, (transactions.apply(sum) >= MIN_COUNT)]
ls_cols = list(df1.columns)
tmpid2cols = {k:v for k,v in enumerate(ls_cols)}

In [ ]:
antecedents = []
for n in range(1, N_POOL+1):
    combos = torch.combinations(torch.tensor(list(tmpid2cols.keys())), r=n)
    antecedents.extend([ set([tmpid2cols[id] for id in c])  for c in combos.numpy()])

df_combo = pd.Series(antecedents, name='x').to_frame()

In [ ]:
def get_count_as_subset_of(candidate_set, ls_refset, ls_len_refset):
    ls_len_unionset = [len(refset.union(candidate_set)) for refset in ls_refset]
    ls_samelen = [1 if u==r else 0 for u, r in zip(ls_len_unionset, ls_len_refset)]

    return sum(ls_samelen)


def get_count(S_candidateset,
        ls_refset,
        batch_size=5000,
):
    S_candidateset = S_candidateset.copy()
    N = len(S_candidateset)

    ls_len_refset = [len(refset) for refset in ls_refset]

    ls = []

    i = 0
    while i < N:
        s_count = S_candidateset[i:i+batch_size].apply(
            lambda x: get_count_as_subset_of(x, ls_refset, ls_len_refset))
        ls.extend(s_count.tolist())
        i += batch_size

    return ls

In [ ]:
df_combo['x_count'] = get_count(df_combo['x'], ls_refset, batch_size=BATCH_SIZE)

In [ ]:
df_combo

,x,x_count
0,{Otu19},13
1,{Otu22},15
2,{Otu25},3
3,{Otu55},16
4,{Otu76},17
...,...,...
10897,"{Otu6755, Otu4606, Otu7254, Otu4514}",1
10898,"{Otu6755, Otu9523, Otu4606, Otu4514}",2
10899,"{Otu7254, Otu9523, Otu4606, Otu4514}",1
10900,"{Otu6755, Otu9523, Otu7254, Otu4514}",3


In [ ]:
N_transactions = len(basket)
print('N_transactions = {}'.format(N_transactions))

df_combo['y'] = [{OTU_NAME}]*df_combo.shape[0]
df_combo['y_count'] = N_transactions

# x_support = probability of x
df_combo['x_support'] = df_combo['x_count'] / N_transactions

# y_support = probability of y
df_combo['y_support']= df_combo['y_count'] / N_transactions

# xy_support = probability of (x and y)
df_combo['xy_support'] = df_combo['y_count'] / N_transactions

# x2y_confidence = p(x and y) / p(x)
df_combo['x2y_confidence'] = df_combo['xy_support'] / df_combo['x_count'] # note y_count = xy_count

# x2y_lifg = p(x and y) / p(x) / p(y)
df_combo['x2y_lift'] = df_combo['xy_support']/df_combo['x_support']/df_combo['y_support']

N_transactions = 25


In [ ]:
df_combo

,x,x_count,y,y_count,x_support,y_support,xy_support,x2y_confidence,x2y_lift
0,{Otu19},13,{Otu11},25,0.52,1.0,1.0,0.076923,1.923077
1,{Otu22},15,{Otu11},25,0.60,1.0,1.0,0.066667,1.666667
2,{Otu25},3,{Otu11},25,0.12,1.0,1.0,0.333333,8.333333
3,{Otu55},16,{Otu11},25,0.64,1.0,1.0,0.062500,1.562500
4,{Otu76},17,{Otu11},25,0.68,1.0,1.0,0.058824,1.470588
...,...,...,...,...,...,...,...,...,...
10897,"{Otu6755, Otu4606, Otu7254, Otu4514}",1,{Otu11},25,0.04,1.0,1.0,1.000000,25.000000
10898,"{Otu6755, Otu9523, Otu4606, Otu4514}",2,{Otu11},25,0.08,1.0,1.0,0.500000,12.500000
10899,"{Otu7254, Otu9523, Otu4606, Otu4514}",1,{Otu11},25,0.04,1.0,1.0,1.000000,25.000000
10900,"{Otu6755, Otu9523, Otu7254, Otu4514}",3,{Otu11},25,0.12,1.0,1.0,0.333333,8.333333


In [ ]:
df_output = df_combo[df_combo['x_count'] >= MIN_COUNT].sort_values(['x2y_lift', 'x_support'], ascending=[False, False])

In [ ]:
fp = '/{}_minCount{}_maxN{}.csv'.format(
    OTU_NAME, MIN_COUNT, N_POOL+1
)
df_output.to_csv(fp)

In [ ]:
files.download(fp)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# VIEW with sorts

In [ ]:
df_output[df_output['x_count'] >= 5].sort_values(['x2y_lift', 'x_support'], ascending=[False, False])

,x,x_count,y,y_count,x_support,y_support,xy_support,x2y_confidence,x2y_lift
11,{Otu471},5,{Otu11},25,0.20,1.0,1.0,0.200000,5.000000
33,"{Otu19, Otu471}",5,{Otu11},25,0.20,1.0,1.0,0.200000,5.000000
43,"{Otu19, Otu7254}",5,{Otu11},25,0.20,1.0,1.0,0.200000,5.000000
54,"{Otu22, Otu471}",5,{Otu11},25,0.20,1.0,1.0,0.200000,5.000000
64,"{Otu22, Otu7254}",5,{Otu11},25,0.20,1.0,1.0,0.200000,5.000000
...,...,...,...,...,...,...,...,...,...
274,"{Otu6755, Otu9523}",18,{Otu11},25,0.72,1.0,1.0,0.055556,1.388889
155,"{Otu9523, Otu97}",19,{Otu11},25,0.76,1.0,1.0,0.052632,1.315789
16,{Otu1828},20,{Otu11},25,0.80,1.0,1.0,0.050000,1.250000
22,{Otu9523},20,{Otu11},25,0.80,1.0,1.0,0.050000,1.250000
